# `mtflib` Tutorial: Advanced Functionality

This notebook explores advanced topics in `mtflib`, including substitution, composition, accessing coefficients, and the use of other elementary functions.

## 1. Initialization

As always, we begin by initializing the global settings for `mtflib`.

In [ ]:
import numpy as np
import sympy as sp
from IPython.display import display
from mtflib import mtf

if not mtf.get_mtf_initialized_status():
    mtf.initialize_mtf(max_order=8, max_dimension=3)
else:
    print("MTF globals are already initialized.")

# Define some variables for our examples
x = mtf.var(1)
y = mtf.var(2)
z = mtf.var(3)

## 2. Substitution and Composition

A powerful feature of `mtflib` is the ability to substitute variables with constants or even other Taylor functions.

### Substitution with a Constant

Replace a variable in a Taylor function with a constant value.

In [ ]:
f_xyz = x + y + z**4
print(f"Original function f(x, y, z):\n{f_xyz}\n")

# Substitute x with 0.1
g_yz = f_xyz.substitute_variable(1, 0.1)
print(f"After substituting x=0.1, we get g(y, z):\n{g_yz}")

### Composition with other Taylor Functions

Substitute variables with other Taylor functions to compose them.

In [ ]:
# Outer function: f(x, y) = x^2 + y^3
f_xy = x**2 + y**3

# Substituting functions:
# g(y) = 1 + y
# h(y, z) = 1 + y*z
g_y = 1 + y
h_yz = 1 + y * z

# Perform composition: f(g(y), h(y,z))
composed_f = f_xy.compose({1: g_y, 2: h_yz})
print(f"Composed function f(g(y), h(y,z)):\n{composed_f}")

## 3. Accessing Coefficients

You can directly access the coefficients and exponents of a Taylor function.

In [ ]:
sin_x = mtf.sin(x)
coefficients = {tuple(exp): coeff for exp, coeff in zip(sin_x.exponents, sin_x.coeffs)}

print("Coefficients of sin(x):")
for exp, coeff in coefficients.items():
    print(f"  Exponent {exp}: {coeff}")

## 4. Other Elementary Functions

`mtflib` supports a variety of other elementary functions.

In [ ]:
print("--- Example: mtf.exp(x+2*y) ---")
x = mtf.var(1)
y = mtf.var(2)
mtf_sin = mtf.sin(x + 2 * y)
sympy_sin_expr = mtf_sin.symprint()
print(
    "The following is a SymPy object, which will render beautifully in a notebook."
)
display(sympy_sin_expr)

# --- Example with ComplexMultivariateTaylorFunction ---
print("--- Example: mtf.exp(i*x) ---")
i = 1j
from mtflib import ComplexMultivariateTaylorFunction

x_complex = ComplexMultivariateTaylorFunction.from_variable(
    var_index=1, dimension=1
)
mtf_complex_exp = mtf.exp(i * x_complex)
sympy_complex_expr = mtf_complex_exp.symprint(symbols=["x"])
print("The following is a SymPy object for a complex function.")
display(sympy_complex_expr)

# --- Example with custom coefficient formatting ---
print("--- Example: Custom coefficient formatting ---")

def custom_formatter(c, p):
    # A simple rational formatter
    if np.iscomplexobj(c):
        return sp.Rational(c.real).limit_denominator(10**p) + sp.I * sp.Rational(
            c.imag
        ).limit_denominator(10**p)
    else:
        return sp.Rational(c).limit_denominator(10**p)

# Re-initialize for the 2D mtf_sin
x = mtf.var(1)
y = mtf.var(2)
mtf_sin = mtf.sin(x + 2 * y)

sympy_custom_format_expr = mtf_sin.symprint(
    precision=3, coeff_formatter=custom_formatter
)
print("The following is a SymPy object with custom rational formatting.")
display(sympy_custom_format_expr)